In [1]:
#Import dependencies
import pandas as pd

In [143]:
#Load the schools data
schools_data = "Resources/schools_complete.csv"
schools_data_df = pd.read_csv(schools_data)

schools_data_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [144]:
#Load students data
students_data = "Resources/students_complete.csv"
students_data_df = pd.read_csv(students_data)

students_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [145]:
#Combine the data
complete_school_data = pd.merge(students_data_df, schools_data_df, on = "school_name", how = "outer" )
complete_school_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#Total schools
total_schools = len(complete_school_data["school_name"].unique())
total_schools

15

In [6]:
#Total students
total_students = len(complete_school_data["Student ID"].unique())
total_students

39170

In [7]:
#Total Budget
total_budget = schools_data_df["budget"].sum()
total_budget

24649428

In [8]:
#Average Math Score
average_maths = complete_school_data["math_score"].mean()
average_maths

78.98537145774827

In [9]:
#Average reading score
average_reading = complete_school_data["reading_score"].mean()
average_reading

81.87784018381414

In [164]:
#* % passing math (the percentage of students who passed math 70 and over)
#complete_school_data[["math_score"] >= 70].count()
math_pass = [x >= 70 for x in complete_school_data["math_score"]]
math_pass_df = sum(math_pass)
math_percent_pass = "{:.2%}".format(math_pass_df/total_students)
math_percent_pass
   

'74.98%'

In [161]:
#% passing reading (the percentage of students who passed reading)
reading_pass = [x >= 70 for x in complete_school_data["reading_score"]]
reading_pass_df = sum(reading_pass)
reading_percent_pass = "{:.2%}".format(reading_pass_df/total_students)
reading_percent_pass

'85.81%'

In [163]:
#% overall passing (the percentage of students who passed math AND reading)
overall_pass = complete_school_data.loc[(complete_school_data["reading_score"] >= 70) & 
                                        (complete_school_data["math_score"] >= 70), :]
overall_pass_percent = "{:.2%}".format(overall_pass["student_name"].count()/total_students)
overall_pass_percent

'65.17%'

In [168]:
district_summary = pd.DataFrame({"Total Schools" : [total_schools],
                                "Total students" : [total_students],
                                "Total budget" : [total_budget],
                                "Average math score" : [average_maths],
                                "Average reading score" : [average_reading],
                                 " % passing math" : [math_percent_pass],
                                "% passing reading" : [reading_percent_pass],
                                "% overall passing" : [overall_pass_percent]})
district_summary

,Total Schools,Total students,Total budget,Average math score,Average reading score,% passing math,% passing reading,% overall passing
0,15,39170,24649428,78.985371,81.87784,74.98%,85.81%,65.17%


### SCHOOL SUMMARY

In [305]:
school_df = complete_school_data[["school_name", "type", "student_name", "budget", "size", "math_score",
                                 "reading_score"]]
#add maths pass & reading pass
school_df["Math_Pass"] = school_df["math_score"] >= 70
school_df["Reading_Pass"] = school_df["reading_score"] >= 70

school_df.head()

C:\Users\SPECTR~1\AppData\Local\Temp/ipykernel_16380/487622926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_df["Math_Pass"] = school_df["math_score"] >= 70
C:\Users\SPECTR~1\AppData\Local\Temp/ipykernel_16380/487622926.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_df["Reading_Pass"] = school_df["reading_score"] >= 70


,school_name,type,student_name,budget,size,math_score,reading_score,Math_Pass,Reading_Pass
0,Huang High School,District,Paul Bradley,1910635,2917,79,66,True,False
1,Huang High School,District,Victor Smith,1910635,2917,61,94,False,True
2,Huang High School,District,Kevin Rodriguez,1910635,2917,60,90,False,True
3,Huang High School,District,Dr. Richard Scott,1910635,2917,58,67,False,False
4,Huang High School,District,Bonnie Ray,1910635,2917,84,97,True,True


In [405]:
#Group Data
school_metrics = school_df.groupby(["school_name", "type"]).mean()
school_metrics["per_student_budget"] = school_metrics["budget"]/school_metrics["size"]
school_metrics["% Math Pass"] = round(school_metrics["Math_Pass"]*100,2)
school_metrics["% Reading Pass"] = round(school_metrics["Reading_Pass"]*100,2)
school_metrics["% Overall Passing"] = round((school_metrics["% Math Pass"] + school_metrics["% Reading Pass"])/2 ,4)
school_metrics

,,budget,size,math_score,reading_score,Math_Pass,Reading_Pass,per_student_budget,% Math Pass,% Reading Pass,% Overall Passing
school_name,type,,,,,,,,,,
Bailey High School,District,3124928.0,4976.0,77.048432,81.033963,0.666801,0.819333,628.0,66.68,81.93,74.305
Cabrera High School,Charter,1081356.0,1858.0,83.061895,83.975780,0.941335,0.970398,582.0,94.13,97.04,95.585
Figueroa High School,District,1884411.0,2949.0,76.711767,81.158020,0.659885,0.807392,639.0,65.99,80.74,73.365
Ford High School,District,1763916.0,2739.0,77.102592,80.746258,0.683096,0.792990,644.0,68.31,79.30,73.805
Griffin High School,Charter,917500.0,1468.0,83.351499,83.816757,0.933924,0.971390,625.0,93.39,97.14,95.265
Hernandez High School,District,3022020.0,4635.0,77.289752,80.934412,0.667530,0.808630,652.0,66.75,80.86,73.805
Holden High School,Charter,248087.0,427.0,83.803279,83.814988,0.925059,0.962529,581.0,92.51,96.25,94.380
Huang High School,District,1910635.0,2917.0,76.629414,81.182722,0.656839,0.813164,655.0,65.68,81.32,73.500
Johnson High School,District,3094650.0,4761.0,77.072464,80.966394,0.660576,0.812224,650.0,66.06,81.22,73.640


In [406]:
#Create DataFrame for the results
school_summary = pd.DataFrame({"Total Students": school_metrics["size"],
                              "Total School Budget":school_metrics["budget"],
                              "Per Student Budget": school_metrics["per_student_budget"],
                              "Average Math Score": school_metrics["math_score"],
                              "Average Reading Score": school_metrics["reading_score"],
                              "% Passing Math": school_metrics["% Math Pass"],
                               "% Passing Reading": school_metrics["% Reading Pass"],
                               "% Overall Passing": school_metrics["% Overall Passing"]})
school_summary


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.68,81.93,74.305
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.13,97.04,95.585
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.99,80.74,73.365
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.31,79.30,73.805
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.39,97.14,95.265
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.75,80.86,73.805
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.51,96.25,94.380
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.68,81.32,73.500
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.06,81.22,73.640


## Highest-Performing Schools

In [407]:
#Top Five Performing
top_five_schools = school_summary.sort_values("% Overall Passing", ascending=False).head(5)
top_five_schools

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.13,97.04,95.585
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.27,97.31,95.290
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.59,95.95,95.270
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.39,97.14,95.265
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.87,96.54,95.205


## Lowest- Performing Schools

In [408]:
#Bottom 5 Performing schools
bottom_five_schools = school_summary.sort_values("% Overall Passing", ascending=True).head(5)
bottom_five_schools

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.37,80.22,73.295
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.99,80.74,73.365
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.68,81.32,73.500
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.06,81.22,73.640
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.31,79.30,73.805


## Math Scores By Grade

In [347]:
math_grade = complete_school_data[["school_name", "grade", "math_score"]]
math_grade.head()

,school_name,grade,math_score
0,Huang High School,9th,79
1,Huang High School,12th,61
2,Huang High School,12th,60
3,Huang High School,12th,58
4,Huang High School,9th,84


In [358]:
school_math_grade = math_grade.groupby(["school_name", "grade"]).mean()
school_math_grade.head()

math_score
school_name         grade            
Bailey High School  10th    76.996772
                    11th    77.515588
                    12th    76.492218
                    9th     77.083676
Cabrera High School 10th    83.154506

In [354]:
math_grade_df = pd.DataFrame({"Maths Score": school_math_grade["math_score"]})
math_grade_df.head()

Maths Score
school_name         grade             
Bailey High School  10th     76.996772
                    11th     77.515588
                    12th     76.492218
                    9th      77.083676
Cabrera High School 10th     83.154506

## Reading Scores By Grade

In [355]:
reading_grade = complete_school_data[["school_name", "grade", "reading_score"]]
reading_grade.head()

,school_name,grade,reading_score
0,Huang High School,9th,66
1,Huang High School,12th,94
2,Huang High School,12th,90
3,Huang High School,12th,67
4,Huang High School,9th,97


In [357]:
school_reading_grade = reading_grade.groupby(["school_name", "grade"]).mean()
school_reading_grade.head()

reading_score
school_name         grade               
Bailey High School  10th       80.907183
                    11th       80.945643
                    12th       80.912451
                    9th        81.303155
Cabrera High School 10th       84.253219

In [359]:
reading_grade_df = pd.DataFrame({"Reading Score": school_reading_grade["reading_score"]})
reading_grade_df.head()

Reading Score
school_name         grade               
Bailey High School  10th       80.907183
                    11th       80.945643
                    12th       80.912451
                    9th        81.303155
Cabrera High School 10th       84.253219

## Scores by School Spending

In [409]:
school_summary


,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.68,81.93,74.305
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.13,97.04,95.585
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.99,80.74,73.365
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.31,79.30,73.805
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.39,97.14,95.265
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.75,80.86,73.805
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.51,96.25,94.380
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.68,81.32,73.500
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.06,81.22,73.640


In [411]:
#Create bins
spending_bins = [0, 580, 600, 620, 640, 660]
spending_labels = ["0-580", "581-600", "601-620","621-640", "641-660"]

school_summary["Per Student Spending"] = pd.cut(school_summary["Per Student Budget"], 
                                                 spending_bins, labels=spending_labels)
#bin_spending= bin_spending.groupby["Per Student Spending"].mean()
bin_spend = school_summary.groupby("Per Student Spending")["Average Math Score",
                                                            "Average Reading Score",
                                                            "% Passing Math",
                                                             "% Passing Reading",
                                                           "% Overall Passing"].mean()
#bin_spend["% Passing Math"] = school_summary ["% Passing Math"]
bin_spend

C:\Users\SPECTR~1\AppData\Local\Temp/ipykernel_16380/1086447756.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bin_spend = school_summary.groupby("Per Student Spending")["Average Math Score",


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Spending,,,,,
0-580,83.274201,83.989488,93.87,96.5400,95.20500
581-600,83.476713,83.867873,93.46,96.4375,94.94875
601-620,83.839917,84.044699,94.59,95.9500,95.27000
621-640,79.474551,82.120471,77.14,87.4680,82.30400
641-660,77.023555,80.957446,66.70,80.6750,73.68750


## Scores by School Size


In [416]:
size_bins = [0, 2000, 4000, 6000]
size_labels = ["small", "medium", "large"]

school_summary["School Size"] = pd.cut(school_summary["Total Students"], 
                                                 size_bins, labels=size_labels)
school_size = school_summary.groupby("School Size")["Average Math Score",
                                                            "Average Reading Score",
                                                            "% Passing Math",
                                                             "% Passing Reading",
                                                    "% Overall Passing"].mean()
school_size

C:\Users\SPECTR~1\AppData\Local\Temp/ipykernel_16380/684611484.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  school_size = school_summary.groupby("School Size")["Average Math Score",


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
small,83.502373,83.883125,93.584286,96.592857,95.088571
medium,78.112137,81.564235,72.044000,83.624000,77.834000
large,77.136883,80.978256,66.496667,81.336667,73.916667


## Scores by School Type

In [426]:
school_type = school_summary.groupby("type")["Average Math Score",
                                                "Average Reading Score",
                                                "% Passing Math",
                                                "% Passing Reading",
                                                "% Overall Passing"].mean()
                                            
school_type

C:\Users\SPECTR~1\AppData\Local\Temp/ipykernel_16380/509404265.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  school_type = school_summary.groupby("type")["Average Math Score",


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.103125
District,76.956733,80.966636,66.548571,80.798571,73.673571


## Observable Trends

1. Charter schools perform better than District schools.
2. Small schools perform better than large schools
3.